In [1]:
from dotenv import load_dotenv

import os

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_APT_KEY')

In [2]:
# OpenAI에 연결

from openai import OpenAI

client = OpenAI(api_key= OPENAI_API_KEY)

In [3]:
# 나이 계산 함수
from datetime import datetime

def  calculate_age(birthdate):
    birth = datetime.strptime(birthdate, "%Y-%m-%d")
    today = datetime.today()
    age = today.year - birth.year - (
        (today.month, today.day) < (birth.month, birth.day)
    )
    return age

In [4]:
# 환율 변환 함수
def convert_currency(amount, from_currency, to_currency, rate):
    return amount * rate

In [5]:
# BMI(체질량지수) 계산 함수

def calculate_bmi(height, weight):
    height_m = height / 100
    bmi = weight / (height_m ** 2)
    return round(bmi, 2)

In [6]:
tools = [
    {
        "type": "function",
        "name": "calculate_age",
        "description": "Calculate age from birthdate",
        "parameters": {
            "type": "object",
            "properties": {
                "birthdate": {"type": "string"}
            },
            "required": ["birthdate"]
        }
    },
    {
        "type": "function",
        "name": "convert_currency",
        "description": "Convert currency with fixed rate",
        "parameters": {
            "type": "object",
            "properties": {
                "amount": {"type": "number"},
                "from_currency": {"type": "string"},
                "to_currency": {"type": "string"},
                "rate": {"type": "number"}
            },
            "required": ["amount", "from_currency", "to_currency", "rate"]
        }
    },
    {
        "type": "function",
        "name": "calculate_bmi",
        "description": "Calculate BMI from height(cm) and weight(kg)",
        "parameters": {
            "type": "object",
            "properties": {
                "height": {"type": "number"},
                "weight": {"type": "number"}
            },
            "required": ["height", "weight"]
        }
    }
]

**BMI(체질량지수) 계산**

In [8]:
input_messages = [{"role": "user", "content": "키 170cm, 몸무게 65kg의 BMI는?"}]

response = client.responses.create(
    model="gpt-4.1",
    input=input_messages,
    tools=tools
)

print('1. ', response.output)

1.  [ResponseFunctionToolCall(arguments='{"height":170,"weight":65}', call_id='call_Yx30zKWBaDTunv0l8j2F07DR', name='calculate_bmi', type='function_call', id='fc_034192707c51520c00696dec87bac081918a0cb4ea765c1b03', status='completed')]


In [9]:
import json

tool_call = response.output[0]
args = json.loads(tool_call.arguments)

In [10]:
result = calculate_bmi(**args)

In [11]:
print(result)

22.49


In [ ]:
input_messages.append(tool_call) 

input_messages.append({                              
"type": "function_call_output",
"call_id": tool_call.call_id,
"output": str(result)
})

response_2 = client.responses.create(
model="gpt-4.1",
input=input_messages,
tools=tools,
)
print(response_2.output_text)



키 170cm, 몸무게 65kg의 BMI는 약 22.5입니다. 이는 정상 체중 범위에 해당합니다.


**환율 변환**

In [21]:
input_messages = [{"role": "user", "content": "100달러를 원화로 바꿔줘 (환율 1330원 적용)"}]

response = client.responses.create(
    model="gpt-4.1",
    input=input_messages,
    tools=tools
)

print('1. ', response.output)

1.  [ResponseFunctionToolCall(arguments='{"amount":100,"from_currency":"USD","to_currency":"KRW","rate":1330}', call_id='call_vJ1yFnsEgyXsI9QbJR8MzRfU', name='convert_currency', type='function_call', id='fc_0fb215ad8668eb9700696deddac4e0819d9277628ef45906ef', status='completed')]


In [22]:
tool_call = response.output[0]
args = json.loads(tool_call.arguments)
result = convert_currency(**args)
print(result)

133000


In [23]:
input_messages.append(tool_call) 

input_messages.append({                               # append result message
"type": "function_call_output",
"call_id": tool_call.call_id,
"output": str(result)
})

response_2 = client.responses.create(
model="gpt-4.1",
input=input_messages,
tools=tools,
)
print(response_2.output_text)

100달러를 환율 1,330원을 적용해서 원화로 바꾸면 133,000원이 됩니다.


**나이 계산**

In [24]:
input_messages = [{"role": "user", "content": "1992-05-01생의 만 나이는?"}]

response = client.responses.create(
    model="gpt-4.1",
    input=input_messages,
    tools=tools
)

print('1. ', response.output)

1.  [ResponseFunctionToolCall(arguments='{"birthdate":"1992-05-01"}', call_id='call_Jh95vus0Gepox6lFrHzW4ljG', name='calculate_age', type='function_call', id='fc_0f24a1be4a8feb5d00696dee27c0c88193af8908b4abfc40f6', status='completed')]


In [26]:
tool_call = response.output[0]
args = json.loads(tool_call.arguments)
result =  calculate_age(**args)
print(result)

33


In [27]:
input_messages.append(tool_call) 

input_messages.append({                               # append result message
"type": "function_call_output",
"call_id": tool_call.call_id,
"output": str(result)
})

response_2 = client.responses.create(
model="gpt-4.1",
input=input_messages,
tools=tools,
)
print(response_2.output_text)

1992년 5월 1일생의 만 나이는 현재 33세입니다.
